<h1 align= center; style="color: white;background-color:black;
         font-size:47px; font-weight:bold;padding:20px;
    font-style:italic;border:3px  solid black;
    box-shadow:2px 2px 5px gray;text-align:center;border-radius:5px;
">   FLIPCART PRODUCT REVIEW DATASET </span></h1>


<div style="display: flex; justify-content: center;">
  <img src="C:\Users\hp\Downloads\FLIOPCART.jpg" 
       style="width: 100%; height: auto; border-radius:10px; background-color:#f0f0f0;">
</div>

<h2 style="background-color:LIGHTBLUE; color:black; text-align:center;
    font-size:30px; font-weight:bold;padding:10px;">IMPORTING LIBRARIES</h1> 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier






<h2 style="background-color:LIGHTBLUE; color:black; text-align:center;
    font-size:30px; font-weight:bold;padding:10px;">READING THE DATASET</h1>

In [2]:
df=pd.read_csv(r"C:\Users\hp\Downloads\Dataset-SA.csv\Dataset-SA.csv")

In [3]:
df

,product_name,product_price,Rate,Review,Summary,Sentiment
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,super!,great cooler excellent air flow and for this p...,positive
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,best budget 2 fit cooler nice cooling,positive
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,the quality is good but the power of air is de...,positive
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,useless product,very bad product its a only a fan,negative
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,ok ok product,neutral
...,...,...,...,...,...,...
205047,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,must buy!,good product,positive
205048,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,super!,nice,positive
205049,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,3,nice,very nice and fast delivery,positive
205050,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,just wow!,awesome product,positive


 <h2 style="background-color:LIGHTBLUE; color:black; text-align:center;
    font-size:30px; font-weight:bold;padding:10px;"> CHECKING SHAPE,INFO,NULL VALUES</h1>

In [4]:
df.shape

(205052, 6)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205052 entries, 0 to 205051
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   product_name   205052 non-null  object
 1   product_price  205052 non-null  object
 2   Rate           205052 non-null  object
 3   Review         180388 non-null  object
 4   Summary        205041 non-null  object
 5   Sentiment      205052 non-null  object
dtypes: object(6)
memory usage: 9.4+ MB


In [6]:
df['review'] = df['Review'] + " " + df['Summary']


In [7]:
df=df.drop(columns=['Review','Summary'],axis=1)

In [8]:
print(df.isnull().sum())


product_name         0
product_price        0
Rate                 0
Sentiment            0
review           24673
dtype: int64


In [9]:
df['review'] = df['review'].fillna('') #Fill Review nulls with empty string


In [10]:
print(df.isnull().sum())


product_name     0
product_price    0
Rate             0
Sentiment        0
review           0
dtype: int64


 <h2 style="background-color:LIGHTBLUE; color:black; text-align:center;
    font-size:30px; font-weight:bold;padding:10px;"> CHECKING UNIQUE VALUES</h1>

In [11]:
print(df['Sentiment'].unique())


['positive' 'negative' 'neutral']


 <h2 style="background-color:LIGHTBLUE; color:black; text-align:center;
    font-size:30px; font-weight:bold;padding:10px;">  MAPPING CATEGORICAL SENTIMENT LABELS TO NUMERIC</h1>

In [12]:
df['Sentiment'] = df['Sentiment'].map({
    'negative': -1,
    'neutral': 0,
    'positive': 1
})


In [13]:
print(df['Sentiment'].unique())
print(df['Sentiment'].value_counts())


[ 1 -1  0]
Sentiment
 1    166581
-1     28232
 0     10239
Name: count, dtype: int64


 <h2 style="background-color:LIGHTBLUE; color:black; text-align:center;
    font-size:30px; font-weight:bold;padding:10px;"> TEXT CLEANING </h1>

In [14]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def clean_review(text):
    tokens = word_tokenize(text)
    cleaned_words = []

    for word in tokens:
        word = word.lower()

        # remove punctuation, stopwords, numbers, short junk
        if (
            word.isalpha() and
            word not in stop_words
        ):
            cleaned_words.append(lemmatizer.lemmatize(word))

    return " ".join(cleaned_words)

In [15]:
df['review'] = df['review'].apply(clean_review)


In [16]:
df

,product_name,product_price,Rate,Sentiment,review
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,1,super great cooler excellent air flow price am...
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,1,awesome best budget fit cooler nice cooling
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,1,fair quality good power air decent
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,-1,useless product bad product fan
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,0,fair ok ok product
...,...,...,...,...,...
205047,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,1,must buy good product
205048,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,1,super nice
205049,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,3,1,nice nice fast delivery
205050,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,1,wow awesome product


In [17]:
x=df['review']
y=df['Sentiment']

In [18]:
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=0.2,
    random_state=42
)

<div style="
    border-left:10px solid #2e7d32;
    padding:10px 14px;color:black; 
    font-size:30px;
    font-weight:800;
    background:#e8f5e9;
">
TF-IDF  VECTORIZATION</div>



In [19]:
vectorizer = TfidfVectorizer( max_features=5000)
x_train_vec = vectorizer.fit_transform(x_train)
x_test_vec = vectorizer.transform(x_test)
model = LinearSVC()
model.fit(x_train_vec, y_train)
y_pred = model.predict(x_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))



Accuracy: 0.9132184048182195


<div style="
    border-left:10px solid #2e7d32;
    padding:10px 14px;color:black; 
    font-size:30px;
    font-weight:800;
    background:#e8f5e9;
">
COUNT  VECTORIZATION</d>

In [20]:
vectorizer1 = CountVectorizer(max_features=5000)
x_train_vec1 = vectorizer1.fit_transform(x_train)
x_test_vec1 = vectorizer1.transform(x_test)
model1 = LinearSVC()
model1.fit(x_train_vec1, y_train)
y_pred1= model1.predict(x_test_vec1)
print("Accuracy:", accuracy_score(y_test, y_pred1))



Accuracy: 0.9109994879422594


<div style="
    border-left:10px solid #2e7d32;
    padding:10px 14px;color:black; 
    font-size:30px;
    font-weight:800;
    background:#e8f5e9;
">
LOGISITIC REGRESSION</d>

In [21]:
model2 = LogisticRegression()
model2.fit(x_train_vec, y_train)
y_pred2 = model2.predict(x_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred2))


Accuracy: 0.915998146838653


<div style="
    border-left:10px solid #2e7d32;
    padding:10px 14px;color:black; 
    font-size:30px;
    font-weight:800;
    background:#e8f5e9;
">
MULTINOMIAL NB </d>

In [22]:
model3 = MultinomialNB()
model3.fit(x_train_vec, y_train)
y_pred3 = model3.predict(x_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred3))


Accuracy: 0.90166053010168


<div style="
    border-left:10px solid #2e7d32;
    padding:10px 14px;color:black; 
    font-size:30px;
    font-weight:800;
    background:#e8f5e9;
">
GRADIENT BOOSTING  </d>

In [23]:
model4 = GradientBoostingClassifier()
model4.fit(x_train_vec, y_train)
y_pred4 = model4.predict(x_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred4))


Accuracy: 0.9090244080856356


<h2 style="background-color:LIGHTBLUE; color:black; text-align:center;
    font-size:30px; font-weight:bold;padding:10px;"> PREDICTION</h1>

In [24]:
reviews = ['ok ok product']
review_vec = vectorizer.transform(reviews)
prediction = model2.predict(review_vec)
label_map = {
    1: "Positive Review 😊",
    0: "Neutral Review 😐",
   -1: "Negative Review 😡"
}
print("Predicted Sentiment:", label_map[prediction[0]])


Predicted Sentiment: Neutral Review 😐
